In [2]:
import pandas as pd
import numpy as np
import datetime
import copy
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.outliers_influence import variance_inflation_factor
import graphviz 
import pydotplus
from sklearn.metrics import roc_curve, auc
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score
from td.tools import * 
from itertools import combinations
import numpy as np

FloatProgress(value=0.0, description='Starting Spark application:', max=2400.0, style=ProgressStyle(descriptio…

SessionID: 4658
YARN Application ID: application_1543970679256_0384



FloatProgress(value=0.0, max=1.0)

SparkSession available as 'spark'.


附加函数,功能说明
save_lab_project(filename),将文件保存到当前Jupyter工程目录中
"savePersonalFile(filename,overwrite=False)",将文件保存到DC“我的资源”中
"open(file,mode='r')",打开本地或者dc文件
load_tlib(spark),加载同盾算法库（默认已加载，算法更新后需重新加载）


FloatProgress(value=0.0, max=1.0)

FloatProgress(value=0.0, max=1.0)

In [ ]:
class split_bins:
    def __init__(self,label,feature,data,bins=5,split=[],min_cnt=20,min_rate=0.05,nan=[],type='ks'):
        self.label = label
        self.feature = feature
        self.data = data
        self.bins = bins
        self.split = set()
        self.min_cnt = min_cnt
        self.min_rate = min_rate
        self.max_iv = -1
        self.nan = nan
        self.type = type

    def cal_main(self,cal_data):
        if (len(cal_data)<self.min_cnt) | (float(len(cal_data))/float(len(self.data))<self.min_rate)| (len(self.split)>5): #(np.std(data[self.feature])/np.mean(data[self.feature])<0.15)
            return np.nan
        else:
            k,value = self.cal_ks(cal_data) 
            if k==1:
               return np.nan
            else: 
                cal_data_left = cal_data[cal_data[self.feature]<value]
                cal_data_right = cal_data[cal_data[self.feature]>value]  
                self.split.update([value])
                re_right = self.cal_main(cal_data_left)
                re_left = self.cal_main(cal_data_right)
    
    def cal_ks(self,data):
        tmp = data[[self.feature,self.label]].sort_values(self.feature,ascending=False)
        one_cnt = sum(tmp[self.label])
        zero_cnt = len(tmp) -one_cnt
        if (one_cnt==0)|(zero_cnt==0):
           return 1,np.nan
        else:
           tmp['one_cum'] = np.cumsum(tmp[self.label])
           tmp['zero_cum'] = np.array([x+1 for x in range(len(tmp))])- np.array(tmp['one_cum'])
           tmp['one_cum'] = tmp['one_cum']/one_cnt
           tmp['zero_cum'] = tmp['zero_cum']/zero_cnt
           tmp['ks_diff'] = map(lambda x,y:abs(x-y),tmp['one_cum'],tmp['zero_cum'])
           
           ks = max(tmp['ks_diff'])
           value = tmp[tmp['ks_diff']==ks][self.feature].iloc[0]
           return ks,value

    def bin_cal_ks(self):
        tmp = self.bin(self.data)
        tmp = tmp[['bin_number',self.label]].sort_values('bin_number',ascending=False)
        
        one_cnt = sum(tmp[self.label])
        zero_cnt = len(tmp) -one_cnt
        
        tmp['one_cum'] = np.cumsum(tmp[self.label])
        tmp['zero_cum'] = np.array([x+1 for x in range(len(tmp))])- np.array(tmp['one_cum'])
        tmp['one_cum'] = tmp['one_cum']/one_cnt
        tmp['zero_cum'] = tmp['zero_cum']/zero_cnt

        tmp['ks_diff'] = map(lambda x,y:abs(x-y),tmp['one_cum'],tmp['zero_cum'])
        
        ks = max(tmp['ks_diff'])
        value = tmp[tmp['ks_diff']==ks]['bin_number'].iloc[0]
        return ks,value


    def bin(self,data):
        tmp = data 
        quan = [round(float(x)/100*len(tmp),0) for x in range(0,100,5)]
        bin_number=0
        bin_split = []
        for i in range(len(quan)-1):
           bin_number = bin_number +1
           bin_split.extend([bin_number]*int(quan[i+1]-quan[i]))
        bin_split.extend([bin_number+1]*int(len(tmp)-quan[i+1]))
        tmp['bin_number'] = bin_split
        return tmp
    
 
    def cal_iv(self,cal_data,is_simple,split):
        if len(split)>0:
           split = sorted(list(split),reverse=False)
           one_cnt = sum(cal_data[self.label])
           zero_cnt = len(cal_data) - one_cnt
           split = sorted(split)
           cal_data['bin'] = map(lambda x:self.bin_name(x,split),cal_data[self.feature])
           piv_table = pd.pivot_table(cal_data,index='bin',values=[self.label],aggfunc=[np.sum,len])
           
           tmp = pd.DataFrame({'bin':list(piv_table.index),'one':list(piv_table['sum'][self.label]),'total':list(piv_table['len'][self.label])})
           tmp['zero'] = tmp['total']-tmp['one']
           tmp['one_ratio'] = tmp['one']/one_cnt
           tmp['zero_ratio'] = tmp['zero']/zero_cnt
           smoothness = min(np.percentile(tmp['one_ratio'],5),np.percentile(tmp['zero_ratio'],5))/100
           tmp['woe'] = map(lambda x,y:-np.log((x+smoothness)/(y+smoothness)),tmp['one_ratio'],tmp['zero_ratio'])
           tmp['iv_sep'] = map(lambda x,y:np.log((x+smoothness)/(y+smoothness))*(x-y),tmp['one_ratio'],tmp['zero_ratio'])
           tmp['iv'] = [sum(tmp['iv_sep'])]*len(tmp)
           tmp['bad_rate'] =  tmp['one']/tmp['total']
           tmp['left_bin'] = [x[1:-1].split(',')[0] for x in tmp['bin']]
           tmp['right_bin'] = [x[1:-1].split(',')[1] for x in tmp['bin']]
            
           tmp['left_bin'] =[float(x) if 'inf' not in x else -np.inf for x in tmp['left_bin']]
           tmp = tmp.sort_values(by='left_bin',ascending=True)
           if is_simple==True:
               return tmp['iv'].iloc[0]
           else:
               return tmp[['bin','one','zero','total','bad_rate','woe','iv_sep','iv','left_bin','right_bin']]
        else:
           one_cnt = sum(cal_data[self.label])
           zero_cnt = len(cal_data) - one_cnt
           return pd.DataFrame({'bin':'NA','one':[one_cnt],'zero':[zero_cnt],'total':[len(cal_data)],'bad_rate':[float(one_cnt)/len(cal_data)],'woe':[np.nan],'iv_sep':[np.nan],'iv':[np.nan]})

    def bin_name(self,x,split):
        split_dev = [1 if x>=i else 0 for i in split]
        try:
          flag = split_dev.index(0)
        except:
          return '['+str(split[len(split)-1])+',inf)'
        
        if flag==0:
           return '(-inf,'+str(split[flag])+')' 
        else:
           return '['+str(split[flag-1])+','+str(split[flag])+')'
    
    def combine_bin(self,reduce_bins):
        combine_list = list(combinations(self.split,reduce_bins-1))
        res = pd.DataFrame()
        for sp in combine_list:
            sp = sorted(sp)
            sp_set = '_'.join([str(x) for x in sp])
            res = res.append(pd.DataFrame({'iv':ss.cal_iv(cal_data=ss.data,is_simple=True,split=sp),'split':[sp_set]}))
        res = res.reset_index(drop = True)
        new_split =  res.sort_values('iv',ascending=False)
        if len(new_split)>0:
            
            for l in range(len(new_split)):
                flag = 1
                s =[float(x) for x in new_split['split'].iloc[l].split('_')]
                for i in range(len(s)):
                    if i==0:
                       if sum(data[self.feature]<s[i])<self.min_cnt:
                          flag = 0
                          break 
                    elif i<(len(s)-1):
                       if sum((data[self.feature]>=s[i-1])&(data[self.feature]<s[i]))<self.min_cnt:
                          flag = 0
                          break 
                    else:
                       if sum(data[self.feature]>=s[i-1])<self.min_cnt:
                          flag = 0
                          break 
                if flag == 1:                                             
                       self.max_iv = new_split['iv'].iloc[l]
                       self.split = set([float(x) for x in new_split['split'].iloc[l].split('_')])
                       return 1 
            return 0
        else:
            return 0
    
    def combine_main(self):
        combine_list = sorted(range(2,min(self.bins,len(self.split)+1)+1),reverse=True)
        for combine_list_i in combine_list:
            res = self.combine_bin(combine_list_i)
            if res==1:
               break
        return None
    
    def cal_bin_tree(self,data):
        parameters = {'min_samples_split':range(10,50,10),'max_depth':[3],'min_samples_leaf':[self.min_cnt]}
        clf = tree.DecisionTreeClassifier()
        gs = GridSearchCV(clf, parameters, cv=5,scoring='roc_auc')
        gs.fit(X=data[[self.feature]],y=data[self.label]) 
        node = gs.best_estimator_.apply(data[[self.feature]])
        ''' 
        tree = DecisionTreeClassifier(min_samples_split=20,min_samples_leaf=self.min_cnt,max_depth=3)
        tree.fit(X=data[[self.feature]],y=data[self.label])
        node = tree.apply(data[[self.feature]])
        '''
        
        tmp = pd.DataFrame({self.feature:data[self.feature],'y':data[self.label],'node':node})
        node_range = tmp[['node',self.feature]].groupby('node')[self.feature].agg(['min','max'])  
        
        self.split = set(node_range['min'].values)
        self.split.update(set(node_range['max'].values))
        #self.split = self.split - set([min(self.split)])
        return None
  
    
    def cal_bin_main(self):
        data_common = self.data[~self.data[self.feature].isin(self.nan)]
        data_uncommon = self.data[self.data[self.feature].isin(self.nan)]  
        if self.type == 'ks' : 
           self.cal_main(data_common)
        elif self.type =='tree':
           self.cal_bin_tree(data_common)
        self.max_iv = self.cal_iv(data_common,is_simple=True,split=self.split)
        self.combine_main()
        iv_common = self.cal_iv(data_common,is_simple=False,split=self.split) 
        
        if len(data_uncommon)>0:
            iv_uncommon = self.cal_iv(data_uncommon,is_simple=False,split=[]) 
            return iv_common.append(iv_uncommon)
        else:
            return iv_common 

In [ ]:
#数据ETL

f = open('/data_d_offline.csv')
#dtypes = recommend_dtypes('data_d_offline.csv')
data_d_offline = pd.read_csv(f,index_col = 0)#,dtype = dtypes)


#标签修正
data_d_offline['label_org'] = data_d_offline['label']
data_d_offline['label'] = [ 1 if x=='Bad' else 0 if x=='Good' else 2 for x in data_d_offline['label_org']]


#
dim = ['m','i']
d_range = ['1day','7day','15day','30day','90day','180day','360day']

for dim_i in dim:
    for d_range_i  in d_range:

        dim_freq_record_Loan_others = dim_i+'_freq_record_Loan'+'_others_'+d_range_i
        dim_cnt_partner_Loan_others = dim_i+'_cnt_partner_Loan'+'_others_'+d_range_i
        
        data_d_offline[dim_freq_record_Loan_others] = data_d_offline[dim_i+'_freq_record_Loan_all_'+d_range_i]
        -data_d_offline[dim_i+'_freq_record_Loan_Imbank_'+d_range_i]-data_d_offline[dim_i+'_freq_record_Loan_P2pweb_'+d_range_i]
        -data_d_offline[dim_i+'_freq_record_Loan_Offloan_'+d_range_i]-data_d_offline[dim_i+'_freq_record_Loan_Consumerfinance_'+d_range_i]
        -data_d_offline[dim_i+'_freq_record_Loan_Bank_'+d_range_i]

        data_d_offline[dim_cnt_partner_Loan_others] = data_d_offline[dim_i+'_cnt_partner_Loan_all_'+d_range_i]
        -data_d_offline[dim_i+'_cnt_partner_Loan_Imbank_'+d_range_i]-data_d_offline[dim_i+'_cnt_partner_Loan_P2pweb_'+d_range_i]
        -data_d_offline[dim_i+'_cnt_partner_Loan_Offloan_'+d_range_i]-data_d_offline[dim_i+'_cnt_partner_Loan_Consumerfinance_'+d_range_i]
        -data_d_offline[dim_i+'_cnt_partner_Loan_Bank_'+d_range_i]

        #data_d_offline = data_d_offline.drop([dim_i+'_freq_record_Loan_all_'+d_range_i,dim_i+'_cnt_partner_Loan_all_'+d_range_i],axis=1)

data_d_offline_part = data_d_offline[(data_d_offline['label']==0)|(data_d_offline['label']==1)]
data_d_offline_part['t1_result'] = data_d_offline_part['t1_result'].astype(str)
data_d_offline_part['t1_result']=[float(x) if x in ['0','1','2','3','4','5','6','7','8','9','10','11','12','13'] else -999 for x in data_d_offline_part['t1_result']]  


data_d_offline_nan = data_d_offline_part.replace(-999,np.nan)
data_d_offline_part0 = data_d_offline_nan.fillna(0)

In [ ]:

def cal_single_rate(dx,nan_list):
    '''
    非null单一值占比
    '''
    max_single_cnt = dx[~dx.isin(nan_list)].value_counts().sort_values(ascending=False).iloc[0]
    return float(max_single_cnt)/(sum(~dx.isin(nan_list))+0.001)


def get_uncor(df,info_select,feature_name,sort_factor,r_value=0.8):
    '''
    相关性
    '''
    save_feature = set()
    cut_feature = set() 
    start_feature= set(df.columns)
    while len(start_feature)>0:  
              x = start_feature.pop() 
              index = df[df[x]>r_value][x].index 
              index = set(index) & start_feature 
              if True:
                    index.update(set([x])) 
                    cal_index = list(index) 
                    save_feature.update([info_select[info_select[feature_name].isin(cal_index)].sort_values(sort_factor,ascending=False)[feature_name].iloc[0]])
                    cut_feature.update(set(cal_index))
                    start_feature = start_feature - set(cal_index) 

    return save_feature


def cal_corr(data,info_select,feature_name,sort_factor,r_value=0.8):
    '''
    通过特征相关性筛选
    '''
    data_dtypes = data.dtypes
    obj_col = [x for x in data.columns if data_dtypes[x]=='O']
    nor_col = [x for x in data.columns if data_dtypes[x]!='O']
    all_feature_key = ['_'.join(x.split('_')[0:-1]) for x in nor_col]
    
    data = data.replace(-999,np.nan)
    data = data.replace(-1111,np.nan)
    #同计算逻辑指标剔除相关性较高
    select = set()
    for x in set(all_feature_key): 
        if ('accept' in x.lower())|('reject' in x.lower())|('review' in x.lower()):
            continue
        else:
            col_x = list(info_select[info_select['feature_key']==x][feature_name])  
            tmp = data[list(set(col_x))] 
            cor = tmp.corr(method='spearman')
            r1 = get_uncor(cor,info_select,feature_name,sort_factor,r_value)
            select.update(set(r1))

            
    #different key
    select_diff = set()
    tmp = data[list(select)] 
    cor = tmp.corr(method='spearman')
    select_diff = get_uncor(cor,info_select,feature_name,sort_factor,r_value)
    return list(select_diff)+obj_col


def cor_label(x,y):
    if x.dtype!='O':
        data=pd.DataFrame({'x':x.values,'y':y.values})
        data = data.dropna()
        return abs(data.corr(method='spearman').iloc[0,1])
    else:
        return 999

def data_pre_processing(data,col_name,label_name,cal_all=0,single_rate=0.5,r_value=0.8,nan_list=[-999]):
    
    if cal_all==0:
        data_single_rate = data[col_name].apply(lambda x :cal_single_rate(x,nan_list),axis=0)
        data_single = data[data_single_rate[data_single_rate<=single_rate].index] 
        info_select = data_single.apply(lambda x:cor_label(x,y=data[label_name]))
        info_select = info_select.to_frame()
        info_select.columns=['rate']
        info_select['feature_name'] = info_select.index
        info_select['feature_key'] = ['_'.join(x.split('_')[0:-1]) for x in info_select['feature_name']]
        
        data_cor_col = cal_corr(data_single,info_select,'feature_name','rate',r_value)
        data_cor = data_single[data_cor_col] 
        
        return data_cor
    else: 
        data_single_rate = data[col_name].apply(lambda x :cal_single_rate(x,nan_list),axis=0) 
        info_select = data[col_name].apply(lambda x:cor_label(x,y=data[label_name]))
        
        result = pd.concat([data_single_rate,info_select],axis=1)
        return result
    
    
def cal_bin(data,factor,feature,bad_name='bad', good_name='good', piece=5, rate=0.005, min_bin_size=50, not_in_list=[-1111,-999,np.nan]):
    
    ss=split_bins(label=factor,feature,data=data,bins=piece,split=[],min_cnt=min_bin_size,min_rate=rate,nan=not_in_list,type='ks')
    tmp = ss.cal_bin_main()
 
    
    if len(tmp)>0:
       tmp['iv_total'] = [sum(tmp['IV'])]*len(tmp)
       tmp['feature_name'] = [feature]*len(tmp) 
       tmp['bad_cover_rate'] = tmp['bad_rate']*tmp['total_count']
       tmp['bad_cover_rate'] =  tmp['bad_cover_rate'].cumsum()
       tmp['bad_cover_rate'] = tmp['bad_cover_rate']*1.0/max(tmp['bad_cover_rate'])
    
       tmp['good_cover_rate'] = (1-tmp['bad_rate'])*tmp['total_count']
       tmp['good_cover_rate'] =  tmp['good_cover_rate'].cumsum()
       tmp['good_cover_rate'] = tmp['good_cover_rate']*1.0/max(tmp['good_cover_rate'])    
    else:
       tmp=pd.DataFrame({'feature_name':[feature]})
    return tmp


def cal_iv(data,factor):
    data_bin = map(lambda x:cal_bin(data,factor,feature=x,bad_name='bad', good_name='good', piece=5, rate=0.005, min_bin_size=50, not_in_list=[-1111,-999,np.nan])
                  ,data.columns)
    return data_bin


def woe_re(x,woe_info,nan_list):
    if x in nan_list:
        return woe_info[woe_info['Bin']=="NA"]['WOE'].iloc[0]
    for i in range(len(woe_info)):
        if (x>woe_info['left_bin'].iloc[i])&(x<=woe_info['right_bin'].iloc[i]):
            return woe_info['WOE'].iloc[i]
    return 'error in find bin!!!'
    
    
def woe_replacement(data,woe_info,col_name,nan_list):
    data_woe=copy.deepcopy(data)
    for col_i in col_name:
        data_woe[col_i]=map(lambda x:woe_re(x,woe_info[woe_info['feature_name']==col_i],nan_list),data_woe[col_i])
    return data_woe[col_name]
        

def select_iv(input_col,iv_info,alpha):
    output_col=[]
    for x in input_col:
        if iv_info[iv_info['feature_name']==x]['iv_total'].iloc[0]>=alpha:
            output_col.append(x)
    return output_col
    

def cal_model(data,col_name,label_data):
    lr_result = pd.DataFrame()
    #Lasso 选入模变量
    for x in [0.001, 0.01, 0.1, 1]:
        tmp_dict = dict()
        parameters = {'C': [x]}  
        clf = LogisticRegression(random_state=0, penalty='l1')
        clf_gs =  GridSearchCV(clf, parameters, cv=5,scoring='roc_auc')
        clf_gs.fit(data_woe_done[sec_select_col],label_data)
      
        lr = LogisticRegression(random_state=0, penalty='l1',C=x)
        lr.fit(data[col_name],label_data)
    
        for k,v in zip(sec_select_col,lr.coef_[0]):
            tmp_dict[k]=v 
        tmp_dict['intercept'] = lr.intercept_[0]
        tmp_dict['paramters']=x
        tmp_dict['mean_test_score'] = clf_gs.cv_results_['mean_test_score']
    
        tmp_dict_df = pd.DataFrame.from_dict(tmp_dict)
        if sum(tmp_dict_df[col_name].iloc[0]<=0)==len(col_name):
            tmp_dict_df['symbol_same_dir'] = [1]
        else:
            tmp_dict_df['symbol_same_dir'] = [0]
        lr_result = lr_result.append(tmp_dict_df)
    return lr_result 



def score_card(bin_info,beta ,score_base=600,pdo=20,odds_base=50):
    '''
    评分卡变量分数
    '''
    n = len(beta.index)-1
    B = round(pdo/np.log(2),2)
    A = round(score_base - B*np.log(odds_base),2)
    intercept = beta['intercept'] if 'intercept' in beta.index else 0
    beta = pd.DataFrame(beta)
    beta.columns=['beta']
    beta['feature_name']=beta.index.values
    bin_info_score = bin_info.merge(beta,left_on='feature_name',right_on='feature_name',how='inner')
    bin_info_score['score'] = A/n-B*(bin_info_score['WOE']*bin_info_score['beta'] + intercept/n)
    bin_info_score['score'] = [round(x,2) for x in bin_info_score['score']]
    return bin_info_score


def cal_score(data,beta ,score_base=600,pdo=20,odds_base=50):
    '''
    分数
    '''
    n = len(beta.index)-1
    B = round(pdo/np.log(2),2)
    A = round(score_base - B*np.log(odds_base),2)
    intercept = beta['intercept'] if 'intercept' in beta.index else 0
    score=pd.DataFrame({'score':[intercept]*len(data)})
    for x in beta.index:
        score['score'] = [score['score'].iloc[i]+data[x].iloc[i]*beta[x] for i in range(len(score))]
    
    score['score'] = A-B*score['score']
    return score

def cal_vif(data,col_name):
    '''
    vif
    '''
    flag=True
    left_col = set(col_name) 
    while flag:  
        vif_value = []
        data_value = data[list(left_col)].values
        for i in range(len(left_col)):
            v = variance_inflation_factor(data_value,i) 
            vif_value.append(v)
        result = pd.DataFrame({'factor':list(left_col),'vif':vif_value})
        result_beta = result[result['vif']>=5].sort_values(by='vif',ascending=False)
        if (len(result_beta)== 0):
            flag = False
        else:
            left_col = left_col -set([result_beta['factor'].iloc[0]]) 
            if len(left_col)==1:
                flag = False
    return list(left_col)

def draw_pr_curve(data,feature,label,beta=0.5):
    if len(set(data[label]))!=2:
        print 'there are not 2 labels in data'
        return None
    data[feature] = data[feature].astype(float)
    tmp_data = data[[feature,label]].sort_values(feature,ascending=False)
    min_value = min(tmp_data[feature])
    max_value = max(tmp_data[feature])
    
    recall = []
    precision = [] 
    f1_score = []
    ngrids = 100
    I_val = []
    ks = []
    tpr = []
    fpr = []
    for i in range(ngrids):
        i_value = max_value-i*(max_value-min_value)/(ngrids-1)
        I_val.append(i_value)
        P = len(tmp_data[tmp_data[label]==1])
        N = len(tmp_data[tmp_data[label]==0])
        TP = len(tmp_data[(tmp_data[feature]>=i_value) & (tmp_data[label]==1)])
        FP = len(tmp_data[(tmp_data[feature]>=i_value) & (tmp_data[label]==0)])
        
        recall.append(TP*1.0/P) 
        precision.append(TP*1.0/(TP+FP)) 
        tpr.append(TP*1.0/P)
        fpr.append(FP*1.0/N)
    f1_score = [(1+beta**2)*recall[i]*precision[i]*1.0/(recall[i]+(1+beta**2)*precision[i]+0.00001) for i in range(len(I_val))]
    ks = [abs(tpr[i]-fpr[i]) for  i in  range(len(I_val))]
    bestks_index = np.argmax(ks)
    bestks = ks[bestks_index]
    
    best_f1_index = np.argmax(f1_score)
    best_f1 = f1_score[best_f1_index]
    
    plt.figure(figsize=(18,10))
    #pr
    except_0 = np.where(np.array(recall)>0)[0].tolist()[0]
    pr_value = [abs(recall[x]-precision[x]) for x in range(except_0,len(recall))]
    balance_index = np.argmin(pr_value)+except_0
    
    plt.subplot(2,2,1)
    plt.title('pr curve')
    plt.plot(recall, precision, 'k',recall, precision, 'o') 
    #plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')   
    plt.legend(['best_ks:('+str(round(bestks,4))+','+str(I_val[bestks_index])+')\n'+'f1_max:('+str(round(best_f1,4))+','+str(I_val[best_f1_index])+')'],loc='lower right')
    plt.ylabel('precision', fontsize=12)
    plt.xlabel('recall', fontsize=12)
    plt.xticks(rotation='vertical') 
    
    pr_df = pd.DataFrame({'ngrids':I_val,'recall':recall,'precision':precision,'f1':f1_score,'ks':ks
                         ,'tpr':tpr})
    #roc
    fpr1,tpr1,threshold = roc_curve(tmp_data[label],tmp_data[feature])
    roc_auc = auc(fpr1,tpr1)
    plt.subplot(2,2,2)
    plt.title('roc curve')
    plt.plot(fpr, tpr, 'k',fpr, tpr,'o') 
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  
    plt.legend( [ 'ROC curve (area = %0.2f)' % roc_auc],loc="lower right")
    
    plt.ylabel('tpr', fontsize=12)
    plt.xlabel('fpr', fontsize=12)
    plt.xticks(rotation='vertical') 
    
    
    #ks 
    plt.subplot(2,2,3)
    plt.title('ks curve')
    plt.plot(range(len(fpr)),fpr,color='blue')  
    plt.plot(range(len(tpr)),tpr,color='blue') 
    plt.plot(range(len(ks)),ks,color='green')
    plt.legend(['fpr','tpr','ks:'+str(bestks)])
     
    plt.subplot(2,2,4)
    p90 = np.percentile(data[feature],95)
    p10 = np.percentile(data[feature],5)
    sns.kdeplot(data[(data[label]==1)&(data[feature]<=p90)&(data[feature]>=p10)][feature], shade=True)
    sns.kdeplot(data[(data[label]==0)&(data[feature]<=p90)&(data[feature]>=p10)][feature], shade=True,color='m')
    plt.legend(['label=1','label=0'])
    
    return pr_df




In [ ]:

X_train, X_test, y_train, y_test = train_test_split(data_d_offline_part, data_d_offline_part['label'], test_size=0.33, random_state=42)
data = X_train

day_range = ['1day','7day','15day','30day','90day','180day','360day']
col_name = [x for x in data.columns if ('day' in x)|('7day' in x)|('15day' in x)]+['t1_result']

data_proc_done = data_pre_processing(data,col_name,'label',cal_all=0,single_rate=0.5,r_value=0.99,nan_list=[-999])

data_bin = map(lambda x:cal_bin(data,factor='label',feature=x,bad_name='bad', good_name='good', piece=5, rate=0.005, min_bin_size=50, not_in_list=[-1111,-999,np.nan])
                  ,data_proc_done.columns)

data_bin_combine = pd.concat(data_bin)

data_bin_combine['left_bin'] = data_bin_combine['left_bin'].astype(float)
data_bin_combine['right_bin'] = data_bin_combine['right_bin'].astype(float)

data_bin_combine['feature_key'] = ['_'.join(x.split('_')[0:-1]) for x in data_bin_combine['feature_name']]

#特征二次筛选-iv

alpha_80 = data_bin_combine['iv_total'].quantile(q=0.8)
alpha_fix = 0.1
iv_select_col = select_iv(data_proc_done.columns,iv_info=data_bin_combine,alpha=alpha_fix)

data_woe_done = woe_replacement(data=data_proc_done,woe_info=data_bin_combine,col_name=iv_select_col,nan_list=nan_list)


#特征二次筛选-相关性
data_cor = data_woe_done.corr(method='spearman')
sec_select_col = cal_corr(data = data_cor,info_select = data_bin_combine ,feature_name = 'feature_name'
                          ,sort_factor = 'iv_total',r_value=0.6)

#vif去除多重共线性
sec_select_col_vif = cal_vif(data = data_woe_done,col_name = sec_select_col)


#Lasso 选入模变量 
param_result = cal_model(data = data_woe_done,col_name=sec_select_col_vif,label_data=y_train)
if sum(param_result['symbol_same_dir']>0)>0:
    score_card_beta = param_result[param_result['symbol_same_dir']>0].sort_values(by=['symbol_same_dir','mean_test_score'],ascending=(False,False))[sec_select_col].iloc[0]
    score_card_beta = score_card_beta[score_card_beta!=0]
    
else:
    print('symbol direction is not same')
    

#评分卡
scorecard_model = score_card(bin_info=data_bin_combine,beta = score_card_beta,score_base=600,pdo=20,odds_base=50)


#评分卡分数
scorecard_result = cal_score(data=data_woe_done,beta=score_card_beta ,score_base=600,pdo=20,odds_base=50)


#绘制评估曲线
plt.switch_backend('agg')
data =pd.DataFrame({'label':y_train.values,'score':max(scorecard_result['score'])-scorecard_result['score']}) 
pr_df = draw_pr_curve(data = data,feature = 'score' ,label = 'label') 

In [ ]:
ss=split_bins(label='label',feature='i_cnt_partner_Loan_all_360day',data=data,bins=5,split=[],min_cnt=20,min_rate=0.05,nan=[-999],type='ks')
ss.cal_bin_main()


In [ ]:
ss=split_bins(label='label',feature='i_cnt_partner_Loan_all_360day',data=data,bins=5,split=[],min_cnt=20,min_rate=0.05,nan=[-999],type='tree')
ss.cal_bin_main()